In [ ]:
"""
cassandara test table schema

CREATE TABLE IF NOT EXISTS crypto.spark_test (
    id varchar,
    low float,
    high float,
    open float,
    close float,
    amount float,
    quantity float,
    tradeCount int,
    startTime bigint,
    closeTime bigint,
    ts_send bigint,
    PRIMARY KEY (id,startTime,closeTime)
  ) WITH default_time_to_live=2592000 and gc_grace_seconds=3600;

"""

In [1]:
import os, sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *


SPARK_VERSION="3.1.1"
SPARK_MASTER_HOST="spark-master"
SPARK_MASTER_PORT="7077"
HIVE_METASTORE_HOST="192.168.10.14"
HIVE_METASTORE_PORT=9083

CASSANDRA_HOST="192.168.10.14"
CASSANDRA_PORT="9042"
CASSANDRA_USERNAME="cassandra"
CASSANDRA_PASSWORD="cassandra"


"""
spark.cores.max: Max number of cores for a spark session, not max cores allocating to each worker.
spark.executor.cores: Number of cores allocating to each worker. The number can be less than number of cores each worker has.
spark.executor.memory: Memory size for each worker.

E.g., 2 cores * 5 workers

    case-1:
        .config("spark.cores.max", 3)
        .config("spark.executor.cores", "1")
        .config("spark.executor.memory", "1g")

        >> 3 executors are created and each executor has 1 core and 1g memory.
        
    case-2:
        .config("spark.cores.max", 3)
        .config("spark.executor.cores", "3")
        .config("spark.executor.memory", "1g")

        >> Failed. 
        >> Since each worker has 2 cores, we can only set even number and less 2.
        >> A executor is created in one spark worker.
    
"""

spark = (
    SparkSession.builder.appName("SparkCassandraApp")
    .config(
        "spark.master",
        "spark://{}:{}".format(SPARK_MASTER_HOST, SPARK_MASTER_PORT),
    )
    .config(
        "spark.hadoop.hive.metastore.uris",
        "thrift://{}:{}".format(HIVE_METASTORE_HOST, HIVE_METASTORE_PORT),
    )
    .config('spark.jars.packages', f'org.apache.spark:spark-sql-kafka-0-10_2.12:{SPARK_VERSION},com.datastax.spark:spark-cassandra-connector_2.12:3.0.0')
    .config("spark.sql.warehouse.dir", "/user/hive/warehouse")
    .config("spark.streaming.stopGracefullyOnShutdown", "true")
    .config("spark.cores.max", 5)
    .config("spark.executor.cores", "1")
    .config("spark.executor.memory", "1g")
    .config("spark.cassandra.connection.host", CASSANDRA_HOST)
    .config("spark.cassandra.connection.port", CASSANDRA_PORT)
    .config("spark.cassandra.auth.username", CASSANDRA_USERNAME)
    .config("spark.cassandra.auth.password", CASSANDRA_PASSWORD)
    .config("spark.debug.maxToStringFields", "100")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9aab6931-8544-489a-9714-384f29526f1d;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.1 in central


:: loading settings :: url = jar:file:/usr/local/spark-3.1.1-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.1 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
	found com.datastax.spark#spark-cassandra-connector_2.12;3.0.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.0.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.7.2 in central
	found com.datastax.oss#native-protocol;1.4.10 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.3.4 in central
	found io.dropwizard.metrics#metrics-core;4.0.5 in central
	found org.hdrhistogram#HdrHistogram;2.1.11 in central
	found org.javatuples#javatuples;1.2 in cent

In [2]:
from pyspark.sql.types import *

"""
{"data": [{"id": ...
"""

# Schema needs to be follow the json data from kafka.
# If a value is string, then its schema must be string.
# E.g., {'id':'123'} -> 'id' is string, not integer.
#       {'id':123} -> 'id' can be integer.
schema = StructType([
    StructField("data", ArrayType(
        StructType([
            StructField("id", StringType(), False),
            StructField("low", StringType(), True),
            StructField("high", StringType(), True),
            StructField("open", StringType(), True),
            StructField("close", StringType(), True),
            StructField("amount", StringType(), True),
            StructField("quantity", StringType(), True),
            StructField("tradeCount", IntegerType(), True),
            StructField("ts_send", LongType(), True),
            StructField("startTime", LongType(), False),
            StructField("closeTime", LongType(), False)
        ])
    ), True)
])
       

# Define Kafka parameters
kafka_bootstrap_servers = "192.168.10.4:9081,192.168.10.4:9082,192.168.10.4:9083"
kafka_topic = "crypto.candles_minute"

# Define options for reading data from Kafka
kafka_source_params = {
    "kafka.bootstrap.servers": kafka_bootstrap_servers,
    "subscribe": kafka_topic,
    "maxOffsetsPerTrigger": 1000,
    "startingOffsets": "latest"  # You can specify where to start reading data from
}

# Read messages from Kafka a topic
kafka_stream_df = spark \
    .readStream \
    .format("kafka") \
    .options(**kafka_source_params) \
    .load()


transformed_df = kafka_stream_df \
    .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("value")) \
    .select(
        element_at(col("value.data.id"), 1).alias("id"),
        element_at(col("value.data.low"), 1).alias("low"),
        element_at(col("value.data.high"), 1).alias("high"),
        element_at(col("value.data.open"), 1).alias("open"),
        element_at(col("value.data.close"), 1).alias("close"),
        element_at(col("value.data.amount"), 1).alias("amount"),
        element_at(col("value.data.quantity"), 1).alias("quantity"),
        element_at(col("value.data.tradeCount"), 1).alias("tradecount"),
        element_at(col("value.data.ts_send"), 1).alias("ts_send"),
        element_at(col("value.data.startTime"), 1).alias("starttime"),
        element_at(col("value.data.closeTime"), 1).alias("closetime")
    )

In [3]:
#transformed_df.writeStream.format("console").start()

In [4]:
# Write the processed data back to Kafka
write_stream = transformed_df \
    .writeStream \
    .outputMode("append") \
    .foreachBatch(lambda df, epoch_id: df.write \
        .format("org.apache.spark.sql.cassandra") \
        .options(table="spark_test", keyspace="crypto") \
        .mode("append") \
        .save()) \
    .start()

24/03/31 14:47:59 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-bf4c20e7-3ffd-4c57-b46e-4d62f2359721. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/31 14:48:04 WARN DefaultTopologyMonitor: [s0] Control node /192.168.10.14:9042 has an entry for itself in system.peers_v2: this entry will be ignored. This is likely due to a misconfiguration; please verify your rpc_address configuration in cassandra.yaml on all nodes in your cluster.
24/03/31 14:48:04 WARN DefaultTopologyMonitor: [s0] Control node /192.168.10.14:9042 has an entry for itself in system.peers_v2: this entry will be ignored. This is likely due to a misconfiguration; please verify your rpc_address configuration in cassandra.yaml on all nodes in your cluster.
24/03/31 14:48:04 WARN SchemaA

24/03/31 14:48:27 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:48:27 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:48:27 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:48:27 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:48:28 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:48:28 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:48:28 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:4

24/03/31 14:48:54 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:48:54 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:48:54 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:48:54 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:48:56 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:48:56 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:48:56 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:4

24/03/31 14:49:29 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:49:29 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:49:29 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:49:29 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:49:30 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:49:30 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:49:30 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:4

24/03/31 14:49:47 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:49:47 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:49:47 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:49:47 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:49:50 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:49:50 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:49:50 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:4

...

24/03/31 14:51:49 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:51:49 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:51:49 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:51:49 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:51:50 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:51:50 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:51:50 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:5

24/03/31 14:52:13 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:52:13 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:52:13 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:52:13 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:52:15 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:52:15 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:52:15 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:5

24/03/31 14:52:34 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:52:34 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:52:34 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:52:34 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:52:34 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:52:34 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:52:34 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:5

24/03/31 14:52:47 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:52:47 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:52:47 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:52:47 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:52:48 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:52:48 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:52:48 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:5

24/03/31 14:53:09 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:53:09 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:53:09 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:53:09 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:53:11 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:53:11 WARN SchemaAgreementChecker: [s0] Unknown peer 55cd3e1c-07a6-468c-960b-597798fc8dd8, excluding from schema agreement check
24/03/31 14:53:11 WARN SchemaAgreementChecker: [s0] Unknown peer 7524df89-ebc3-4dad-a63a-8f192aa09c9d, excluding from schema agreement check
24/03/31 14:5